# 오늘은 LeNet 구조를 만들어봅시다


LeNet 구조는 CNN이며, 초기에 만들어진 모델입니다. 

2가지 모델(Sigmoid, ReLU)를 만들어 두 모델의 성능을 비교해봅시다.


## 1.우선 필요 라이브러리를 import 합니다.

In [1]:
import numpy as np
import matplotlib.pyplot as plt

import torch
from torchvision import datasets
import torchvision.transforms as transforms

import torch.optim as optim

import ssl
ssl._create_default_https_context = ssl._create_unverified_context

## 2. 딥러닝 모델을 설계할 때 활용하는 장비 확인

In [2]:
if torch.cuda.is_available():
    device = torch.device('cuda')
else:
    device = torch.device('cpu')

print('Using PyTorch version:', torch.__version__, ' Device:', device)

Using PyTorch version: 1.12.1  Device: cpu


## 3. MNIST 데이터 다운로드 

 1. Training data와 Test data 분리하기
 
 2. Training data를 Training data 와 Validation data로 분리하기

In [3]:
BATCH_SIZE = 64

transform = transforms.Compose(
    [
     transforms.ToTensor(),
     transforms.Normalize(0.1307, 0.3081), #채우세요),
    ])

train_data = datasets.MNIST(root='MNIST_data', train=True, download=True, transform=transform)#채우세요
test_data = datasets.MNIST(root='MNIST_data', train=False, download=True, transform=transform)
#채우세요

train, val = torch.utils.data.random_split(train_data, [int(0.8 * len(train_data)), len(train_data) - int(0.8 * len(train_data))]) #채우세요)


train_loader = torch.utils.data.DataLoader(train, batch_size=BATCH_SIZE, shuffle=True) #채우세요
val_loader = torch.utils.data.DataLoader(val, batch_size=BATCH_SIZE, shuffle=True) #채우세요
test_loader = torch.utils.data.DataLoader(test_data, batch_size=BATCH_SIZE, shuffle=True)#채우세요


## 4. torch.nn을 이용하여 모델-1 만들기

   1) 아래의 그림 중 LeNet 구조를 구현 할 것
   
   2) Sigmoid 활성화 함수를 이용할 것
   
   
![](Comparison_image_neural_networks.svg.png)

In [10]:
import torch.nn as nn

class Model_1(nn.Module):
    def __init__(self):
        super(Model_1, self).__init__()
        self.conv1 = nn.Conv2d(1,6,5, padding=2)
        self.sigmoid1 = nn.Sigmoid()
        self.pool1 = nn.AvgPool2d(2, stride=2)
        
        self.conv2 = nn.Conv2d(6, 16, 5)
        self.sigmoid2 = nn.Sigmoid()
        self.pool2 = nn.AvgPool2d(2, stride=2)
        
        self.fc1 = nn.Linear(16*5*5, 120)
        self.sigmoid3 = nn.Sigmoid()
        
        self.fc2 = nn.Linear(120, 84)
        self.sigmoid4 = nn.Sigmoid()
        
        self.fc3 = nn.Linear(84, 10)
        #채우세요
        
    def forward(self,x):
        x = self.pool1(self.sigmoid1(self.conv1(x)))
        x = self.pool2(self.sigmoid2(self.conv2(x)))
        
        x = x.view(-1, 16*5*5)
        
        x = self.sigmoid3(self.fc1(x))
        x = self.sigmoid4(self.fc2(x))
        x = self.fc3(x)
        #채우세요
        
        return x

## 5. torch.nn을 이용하여 모델-2 만들기

   LeNet 모델에서 ReLU 활성화 함수를 사용하시요

In [11]:
class Model_2(nn.Module):
    def __init__(self):
        super(Model_2, self).__init__()
        self.conv1 = nn.Conv2d(1, 6, 5, padding=2)
        self.relu1 = nn.ReLU()
        self.pool1 = nn.AvgPool2d(2, stride=2)
        
        self.conv2 = nn.Conv2d(6, 16, 5)
        self.relu2 = nn.ReLU()
        self.pool2 = nn.AvgPool2d(2, stride=2)
        
        self.fc1 = nn.Linear(16*5*5, 120)
        self.relu3 = nn.ReLU()
        
        self.fc2 = nn.Linear(120, 84)
        self.relu4 = nn.ReLU()
        
        self.fc3 = nn.Linear(84, 10)
        #채우세요
        
    def forward(self,x):
        x = self.pool1(self.relu1(self.conv1(x)))
        x = self.pool2(self.relu2(self.conv2(x)))
        
        x = x.view(-1, 16*5*5)
        
        x = self.relu3(self.fc1(x))
        x = self.relu4(self.fc2(x))
        
        x = self.fc3(x)
        #채우세요
        
        return x

## 7. 학습 준비하기

1) 1 epoch를 학습할 수 있는 함수 만들기

2) Test와 Validation data의 정확도 계산할 수 있는 함수 만들기

In [16]:
def training_epoch(train_loader, network, loss_func, optimizer, epoch):
    train_losses = []
    train_correct = 0
    log_interval = 300
    
    for batch_idx, (image, label) in enumerate(train_loader):
        image, label = image.to(device), label.to(device)

        # 미분값의 초기화
        optimizer.zero_grad()
        #채우세요

        # Forward propagration 계산하기.
        outputs = network(image)#채우세요
        
        
        # Cross_entropy 함수를 적용하여 loss를 구하고 저장하기
        loss = loss_func(outputs, label)#채우세요
        train_losses.append(loss.item())

        # training accuracy 정확도 구하기 위해 맞는 샘플 개수 세기
        pred = outputs.data.max(1, keepdim=True)[1]#채우세요
        train_correct += pred.eq(label.data.view_as(pred)).sum()#채우세요

        # Gradinet 구하기
        loss.backward()
        #채우세요

        # weight값 update 하기
        optimizer.step()
        #채우세요

        # 학습 상황 출력
        if batch_idx % log_interval == 0:
            print('Train Epoch: {} [{}/{} ({:.2f}%)]\tLoss: {:.6f}'
                  .format(epoch, batch_idx * len(label), len(train_loader.dataset),100. * batch_idx / len(train_loader),
                          loss.item()))
            
    return train_losses, train_correct

In [17]:
def test_epoch(test_loader, network, loss_func, val = False):
    correct = 0
    
    test_losses = []
    
    with torch.no_grad():
        for batch_idx, (image, label) in enumerate(test_loader):
            image, label = image.to(device), label.to(device)

            # Forward propagration 계산하기.
            outputs = network(image)#채우세요

            # Cross_entropy 함수를 적용하여 loss를 구하기
            loss = loss_func(outputs, label)#채우세요
            test_losses.append(loss.item())

            # Batch 별로 정확도 구하기
            pred = outputs.data.max(1, keepdim=True)[1]#채우세요
            correct += pred.eq(label.data.view_as(pred)).sum()
#채우세요

        # 전체 정확도 구하기
        test_accuracy = 100. * correct / len(test_loader.dataset)

        #중간결과 출력
        if val is True:
                print('Validation set: Accuracy: {}/{} ({:.2f}%)\n'
              .format(correct, len(test_loader.dataset),100. * correct / len(test_loader.dataset)))
        
        else:
            print('Test set: Accuracy: {}/{} ({:.2f}%)\n'
                  .format(correct, len(test_loader.dataset),100. * correct / len(test_loader.dataset)))
        
    return test_losses, test_accuracy


## 8. 위 정의된 함수로 학습 함수 만들기

Adam Optimizer를 사용하여 학습시키기

In [18]:
def training(network, learning_rate = 0.001):
    
    epoches = 15
    
    cls_loss = nn.CrossEntropyLoss() #채우세요
    optimizer = optim.Adam(network.parameters(), lr=learning_rate)#채우세요
    
    train_losses_per_epoch = []
    test_losses_per_epoch = []
    
    train_accuracies = []
    test_accuracies = []
    
    
    for epoch in range(epoches):
                
        # 모델를 학습 중이라고 선언하기
        network.train()
        
        train_losses, train_correct = training_epoch(train_loader,network,cls_loss,optimizer, epoch)
        
        # epoch 별로 loss 평균값, 정확도 구하기
        average_loss = np.mean(train_losses)
        train_losses_per_epoch.append(average_loss)
        
        train_accuracy = train_correct / len(train_loader.dataset) * 100
        train_accuracies.append(train_accuracy)
        
        # epoch 별로 정확도 출력
        print('\nTraining set: Accuracy: {}/{} ({:.2f}%)'
              .format(train_correct, len(train_loader.dataset),100. * train_correct / len(train_loader.dataset)))

        
        ### 학습 중에 test 결과 보기
        
        # 모델 test 중인 것을 선언하기
        network.eval()
        
        correct = 0
        with torch.no_grad():
            test_losses, test_accuracy = test_epoch(val_loader, network, cls_loss, True)

        test_losses_per_epoch.append(np.mean(test_losses))
        test_accuracies.append(test_accuracy)
        
    with torch.no_grad():
        test_losses, test_accuracy = test_epoch(test_loader, network, cls_loss, False)
        
    return train_losses_per_epoch, test_losses_per_epoch, train_accuracies, test_accuracies


In [19]:
network = Model_1().to(device)
rlt_const = training(network)

[W NNPACK.cpp:51] Could not initialize NNPACK! Reason: Unsupported hardware.


Train Epoch: 0 [0/48000 (0.00%)]	Loss: 2.373318
Train Epoch: 0 [19200/48000 (40.00%)]	Loss: 0.885212
Train Epoch: 0 [38400/48000 (80.00%)]	Loss: 0.315022

Training set: Accuracy: 32280/48000 (67.25%)
Validation set: Accuracy: 10833/12000 (90.28%)

Train Epoch: 1 [0/48000 (0.00%)]	Loss: 0.516416
Train Epoch: 1 [19200/48000 (40.00%)]	Loss: 0.207211
Train Epoch: 1 [38400/48000 (80.00%)]	Loss: 0.340304

Training set: Accuracy: 44263/48000 (92.21%)
Validation set: Accuracy: 11232/12000 (93.60%)

Train Epoch: 2 [0/48000 (0.00%)]	Loss: 0.328751
Train Epoch: 2 [19200/48000 (40.00%)]	Loss: 0.316777
Train Epoch: 2 [38400/48000 (80.00%)]	Loss: 0.336372

Training set: Accuracy: 45518/48000 (94.83%)
Validation set: Accuracy: 11496/12000 (95.80%)

Train Epoch: 3 [0/48000 (0.00%)]	Loss: 0.219351
Train Epoch: 3 [19200/48000 (40.00%)]	Loss: 0.056791
Train Epoch: 3 [38400/48000 (80.00%)]	Loss: 0.061991

Training set: Accuracy: 46097/48000 (96.04%)
Validation set: Accuracy: 11563/12000 (96.36%)

Train Ep

In [20]:
network = Model_2().to(device)
rlt_const = training(network)

Train Epoch: 0 [0/48000 (0.00%)]	Loss: 2.320723
Train Epoch: 0 [19200/48000 (40.00%)]	Loss: 0.399224
Train Epoch: 0 [38400/48000 (80.00%)]	Loss: 0.139076

Training set: Accuracy: 43489/48000 (90.60%)
Validation set: Accuracy: 11616/12000 (96.80%)

Train Epoch: 1 [0/48000 (0.00%)]	Loss: 0.028750
Train Epoch: 1 [19200/48000 (40.00%)]	Loss: 0.028404
Train Epoch: 1 [38400/48000 (80.00%)]	Loss: 0.085736

Training set: Accuracy: 46788/48000 (97.47%)
Validation set: Accuracy: 11760/12000 (98.00%)

Train Epoch: 2 [0/48000 (0.00%)]	Loss: 0.017078
Train Epoch: 2 [19200/48000 (40.00%)]	Loss: 0.176875
Train Epoch: 2 [38400/48000 (80.00%)]	Loss: 0.034385

Training set: Accuracy: 47079/48000 (98.08%)
Validation set: Accuracy: 11763/12000 (98.03%)

Train Epoch: 3 [0/48000 (0.00%)]	Loss: 0.046184
Train Epoch: 3 [19200/48000 (40.00%)]	Loss: 0.044332
Train Epoch: 3 [38400/48000 (80.00%)]	Loss: 0.111930

Training set: Accuracy: 47293/48000 (98.53%)
Validation set: Accuracy: 11801/12000 (98.34%)

Train Ep

## 9. 두모델의 성능을 비교하시오

정답)relu 모델이 더 좋다고 볼 수 잇음. 첫 번째 모델보다 더 높은 정확도를 보이고, loss도 더 작기 때문이다.

In [21]:
print("이서영 2020125046")

이서영 2020125046
